In [12]:
import pandas as pd
import re
from datetime import date
import numpy as np
import math
from nltk import everygrams
import nltk

In [13]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/admin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
FILE_NAME = 'internshala-cs-internships.csv'
JUNE_FILE_NAME = 'internshala-cs-internships-june.csv'
CSV_PATH = f'../../Scrapping/{JUNE_FILE_NAME}'

df = pd.read_csv(CSV_PATH)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1213 entries, 0 to 1212
Data columns (total 20 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   Unnamed: 0                               1213 non-null   int64 
 1   location                                 1213 non-null   object
 2   domain_profile                           1213 non-null   object
 3   company_name                             1213 non-null   object
 4   about_the_work_from_home_job/internship  818 non-null    object
 5   skill(s)_required                        1070 non-null   object
 6   who_can_apply                            1213 non-null   object
 7   perks                                    1154 non-null   object
 8   number_of_openings                       1213 non-null   int64 
 9   company_details                          1213 non-null   object
 10  start_date                               1213 non-null   obj

### Potentially Important Fields

+ location
+ domain_profile
+ company_name
+ stipend
+ skill(s)_required
+ number_of_openings
+ applicants
+ is_part_time
+ duration
+ start_data (maybe complicated to deal with)
+ apply_by (for EDA purpose)


#### Cleaning Required for
+ skills
+ start_data
+ stipend
+ apply_by
+ applicants

In [15]:
del df['Unnamed: 0']
df.head(3)

,location,domain_profile,company_name,about_the_work_from_home_job/internship,skill(s)_required,who_can_apply,perks,number_of_openings,company_details,start_date,duration,stipend,apply_by,applicants,other_requirements,about_the_internship,additional_information,is_part_time,about_part_time_job/internship
0,Work From Home,Web Development,Perfect Skills,Selected intern's day-to-day responsibilities ...,PHP\nHTML\nCSS\nJavaScript\nPython\nWordPress,Only those candidates can apply who:\n1. are a...,Certificate\n Letter of recommendation\n Flexi...,2,Website;Perfect Skills is committed to helping...,Starts immediatelyImmediately,4 Months,5000 /month,14 Jul' 21,46 applicants,NaN,NaN,NaN,0,NaN
1,Work From Home,Web Development,Geekster,Selected intern's day-to-day responsibilities ...,HTML\nCSS\nJavaScript\nAngularJS\nNode.js\nRea...,Only those candidates can apply who:\n1. are a...,Certificate\n Flexible work hours,1,Website;Geekster is on a mission to redefine t...,Starts immediatelyImmediately,3 Months,15000 /month,13 Jul' 21,53 applicants,1. Should have hands-on experience with Node.j...,NaN,NaN,0,NaN
2,Work From Home,Backend Development,Global Free School,Selected intern's day-to-day responsibilities ...,PHP\nMySQL\nHTML\nCSS\nJavaScript\njQuery\nCod...,Only those candidates can apply who:\n1. are a...,Certificate\n Letter of recommendation,2,Website;We are a team of IIT graduates working...,Starts immediatelyImmediately,6 Months,10000 /month,4 Jul' 21,54 applicants,NaN,NaN,NaN,0,NaN


### Skills

In [16]:
# rename skill(s)_required to skills
_ = df.rename(columns={"skill(s)_required":'skills'}, inplace=True)

In [17]:
skills_super_set = dict()

def agg_skills_counter(input_string):
    if(type(input_string) == str):
        for i in input_string.split('\n'):
            skills_super_set[i] = skills_super_set.get(i, 0) + 1
    return None

_ = df.skills.agg(agg_skills_counter)

In [18]:
skills_super_set

{'PHP': 189,
 'HTML': 294,
 'CSS': 267,
 'JavaScript': 335,
 'Python': 149,
 'WordPress': 128,
 'AngularJS': 62,
 'Node.js': 176,
 'ReactJS': 188,
 'MySQL': 102,
 'jQuery': 60,
 'CodeIgniter': 29,
 'NoSQL': 9,
 'LARAVEL': 61,
 'REST API': 106,
 'SQLite': 8,
 'React Native': 103,
 'Android': 105,
 'SQL': 63,
 'MongoDB': 88,
 'C++ Programming': 39,
 'Java': 146,
 'Angular 7.0': 43,
 'ASP.NET': 21,
 'MS SQL Server': 25,
 'Amazon Web Services (AWS)': 36,
 'Blockchain': 7,
 'Unity 3D': 14,
 'Unity Engine': 13,
 '3ds Max': 2,
 'Autodesk Maya': 3,
 'Unreal Engine': 7,
 'Xcode': 3,
 'Swift': 17,
 'iOS': 41,
 'Vue Js': 21,
 'MEAN/MERN Stack': 20,
 'CMS': 6,
 'Flutter': 79,
 'Dart': 23,
 'Adobe Photoshop': 27,
 'UI & UX Design': 38,
 'GitHub': 24,
 'J2EE': 6,
 'Spring MVC': 17,
 'ARM Microcontroller': 2,
 'Embedded Systems': 8,
 'Arduino': 5,
 'C Programming': 30,
 'Internet of Things (IoT)': 16,
 'Google Cloud Computing': 8,
 'Bootstrap': 77,
 'Angular 2.0': 16,
 'Firebase': 53,
 'Firebase Clou

In [19]:
count_list = {}
for skill in skills_super_set:
    if(skill):
        value = skills_super_set[skill]
        temp = []
        count_list.get(value, temp).append(skill)
        if(len(temp) > 0):
            count_list[value] = temp

In [20]:
n_skills = {}
for c in count_list:
    n_skills[c] = len(count_list[c])

In [21]:
df

,location,domain_profile,company_name,about_the_work_from_home_job/internship,skills,who_can_apply,perks,number_of_openings,company_details,start_date,duration,stipend,apply_by,applicants,other_requirements,about_the_internship,additional_information,is_part_time,about_part_time_job/internship
0,Work From Home,Web Development,Perfect Skills,Selected intern's day-to-day responsibilities ...,PHP\nHTML\nCSS\nJavaScript\nPython\nWordPress,Only those candidates can apply who:\n1. are a...,Certificate\n Letter of recommendation\n Flexi...,2,Website;Perfect Skills is committed to helping...,Starts immediatelyImmediately,4 Months,5000 /month,14 Jul' 21,46 applicants,NaN,NaN,NaN,0,NaN
1,Work From Home,Web Development,Geekster,Selected intern's day-to-day responsibilities ...,HTML\nCSS\nJavaScript\nAngularJS\nNode.js\nRea...,Only those candidates can apply who:\n1. are a...,Certificate\n Flexible work hours,1,Website;Geekster is on a mission to redefine t...,Starts immediatelyImmediately,3 Months,15000 /month,13 Jul' 21,53 applicants,1. Should have hands-on experience with Node.j...,NaN,NaN,0,NaN
2,Work From Home,Backend Development,Global Free School,Selected intern's day-to-day responsibilities ...,PHP\nMySQL\nHTML\nCSS\nJavaScript\njQuery\nCod...,Only those candidates can apply who:\n1. are a...,Certificate\n Letter of recommendation,2,Website;We are a team of IIT graduates working...,Starts immediatelyImmediately,6 Months,10000 /month,4 Jul' 21,54 applicants,NaN,NaN,NaN,0,NaN
3,Mumbai,Web Development,Knitroot India Private Limited,NaN,WordPress,Only those candidates can apply who:\n1. are a...,Certificate\n Letter of recommendation\n Flexi...,1,Website;We specialize in occasion-based mercha...,Starts immediatelyImmediately,1 Month,5000 /month,13 Jul' 21,38 applicants,1. Candidates who have experience of 1 year in...,Selected intern's day-to-day responsibilities ...,NaN,0,NaN
4,Work From Home,Web Development,FitnessForce,Selected intern's day-to-day responsibilities ...,Python\nNode.js\nReactJS,Only those candidates can apply who:\n1. are a...,Letter of recommendation\n Flexible work hours...,2,Website;FitnessForce is a SaaS-based CRM club ...,Starts immediatelyImmediately,6 Months,10000 /month,5 Jul' 21,66 applicants,1. Looking for a fresher computer programmer i...,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,Mumbai,Odoo Development,Best Roadways Limited,NaN,Odoo,Only those candidates can apply who:\n1. are a...,Certificate,1,Website;Best Roadways was founded in 1985 with...,Starts immediatelyImmediately,3 Months,10000 /month,2 Jul' 21,Be an early applicant,1. Must have strong knowledge of Odoo,Selected intern's day-to-day responsibilities ...,NaN,0,NaN
1209,Work From Home,Web Development,Dot2Globe Solutions India Private Limited,Selected intern's day-to-day responsibilities ...,HTML\nCSS\nJavaScript\nUI & UX Design\nFigma,Only those candidates can apply who:\n1. are a...,Certificate\n 5 days a week,1,Website;Dot2Globe Solutions is a start-up orga...,Starts immediatelyImmediately,3 Months,5000-8000 /month,14 Jul' 21,29 applicants,1. Must have worked as UI/UX design intern (wi...,NaN,Note: Extension of internship/offer of full-ti...,0,NaN
1210,Work From Home,Web Development,TikiLIVE,Selected intern's day-to-day responsibilities ...,NaN,Only those candidates can apply who:\n1. are a...,Certificate\n Letter of recommendation,3,Website;We make OTT delivery solutions worldwi...,Starts immediatelyImmediately,1 Month,20000 /month,2 Jul' 21,391 applicants,NaN,NaN,NaN,1,NaN
1211,Work From Home,Web Development,Success Genix,Selected intern's day-to-day responsibilities ...,HTML\nCSS\nREST API\nReactJS\nAmazon Web Serve...,Only those candidates can apply who:\n1. are a...,Certificate\n Letter of recommendation\n Flexi...,2,Success Genix is a team of people that make ou...,Starts immediatelyImmediately,2 Months,3000 /month,6 Jul' 21,100 applicants,NaN,NaN,NaN,1,NaN


In [22]:
ohe_dummies = df.skills.str.get_dummies(sep='\n')

df = df.join(ohe_dummies)

### start_date

In [23]:
df.start_date[:10]

0    Starts immediatelyImmediately
1    Starts immediatelyImmediately
2    Starts immediatelyImmediately
3    Starts immediatelyImmediately
4    Starts immediatelyImmediately
5    Starts immediatelyImmediately
6    Starts immediatelyImmediately
7    Starts immediatelyImmediately
8    Starts immediatelyImmediately
9    Starts immediatelyImmediately
Name: start_date, dtype: object

In [24]:
df.start_date.unique()

array(['Starts\xa0immediatelyImmediately', "5 Aug - 10 Aug' 21",
       "3 Aug - 31 Aug' 21"], dtype=object)

In [25]:
# Replace 'Starts immediatelyImmediately' with 'immediately'

df.start_date = df.start_date.apply(lambda s: 'Immediately' if s == 'Starts\xa0immediatelyImmediately' else np.nan)

### Stipend

In [26]:
df.stipend.value_counts()

5000 /month                  141
10000 /month                  94
1000 /month                   88
2000 /month                   50
8000 /month                   50
                            ... 
1200-6200 /month               1
6500 /month +  Incentives      1
3500-5000 /month               1
1599 /month                    1
7000-9000 /month               1
Name: stipend, Length: 151, dtype: int64

In [27]:
def remove_month(s):
    split = s.split(" ")
    return f'{split[0]}'
        
def convert_package_to_min_salaries(salary):
    if(salary == 'Unpaid'):
        return 0
    if(salary.find('-') != -1):
        sal_1, sal_2 = salary.split('-')
        integer_sal_1 = int(sal_1)
        integer_sal_2 = int(sal_2)
        min_sal = min(integer_sal_1, integer_sal_2)
        return min_sal
    else:
        return int(salary)

def convert_package_to_max_salaries(salary):
    if(salary == 'Unpaid'):
        return 0
    if(salary.find('-') != -1):
        sal_1, sal_2 = salary.split('-')
        integer_sal_1 = int(sal_1)
        integer_sal_2 = int(sal_2)
        max_sal = max(integer_sal_1, integer_sal_2)
        return max_sal
    else:
        return int(salary)

def convert_package_to_mean_salaries(salary):
    if(salary == 'Unpaid'):
        return 0
    if(salary.find('-') != -1):
        sal_1, sal_2 = salary.split('-')
        integer_sal_1 = int(sal_1.replace(',',''))
        integer_sal_2 = int(sal_2.replace(',',''))
        mean_sal = np.average([integer_sal_1, integer_sal_2])
        return mean_sal
    else:
        return int(salary)


# min_salary_feature = df.stipend.apply(convert_package_to_min_salaries)
# max_salary_feature = df.stipend.apply(convert_package_to_max_salaries)
# mean_salary_feature = df.stipend.apply(convert_package_to_mean_salaries)
    
df.stipend = df.stipend.apply(remove_month)
df['min_stipend'] = df.stipend.apply(convert_package_to_min_salaries)
df['max_stipend'] = df.stipend.apply(convert_package_to_max_salaries)
df['mean_stipend'] = df.stipend.apply(convert_package_to_mean_salaries)

In [28]:
df.head(3)

,location,domain_profile,company_name,about_the_work_from_home_job/internship,skills,who_can_apply,perks,number_of_openings,company_details,start_date,...,Wireframing,WordPress,XML,Xamarin,Xcode,iOS,jQuery,min_stipend,max_stipend,mean_stipend
0,Work From Home,Web Development,Perfect Skills,Selected intern's day-to-day responsibilities ...,PHP\nHTML\nCSS\nJavaScript\nPython\nWordPress,Only those candidates can apply who:\n1. are a...,Certificate\n Letter of recommendation\n Flexi...,2,Website;Perfect Skills is committed to helping...,Immediately,...,0,1,0,0,0,0,0,5000,5000,5000.0
1,Work From Home,Web Development,Geekster,Selected intern's day-to-day responsibilities ...,HTML\nCSS\nJavaScript\nAngularJS\nNode.js\nRea...,Only those candidates can apply who:\n1. are a...,Certificate\n Flexible work hours,1,Website;Geekster is on a mission to redefine t...,Immediately,...,0,0,0,0,0,0,0,15000,15000,15000.0
2,Work From Home,Backend Development,Global Free School,Selected intern's day-to-day responsibilities ...,PHP\nMySQL\nHTML\nCSS\nJavaScript\njQuery\nCod...,Only those candidates can apply who:\n1. are a...,Certificate\n Letter of recommendation,2,Website;We are a team of IIT graduates working...,Immediately,...,0,0,0,0,0,0,1,10000,10000,10000.0


### applicants

In [29]:
# df.applicants.unique()

In [30]:
def remove_applicants(s):
    test_str = str(s)
    re_extract_num = re.compile('\d+')
    output = re_extract_num.findall(test_str)
    if(len(output) != 1):
        return 0
    else:
        return output[0]

df.applicants = df.applicants.apply(remove_applicants)


In [31]:
df.applicants = pd.to_numeric(df.applicants)

### Save to disk

In [32]:
df.to_csv('internshala_clean_june.csv', index=False)

### Location

Does not require any additional cleaning

In [33]:
number_of_locations = len(df.location.unique())
print(f"Number of Unique Location {number_of_locations}")

df.location.value_counts()[:20]

Number of Unique Location 53


Work From Home    818
Delhi              65
Bangalore          37
Mumbai             36
Pune               28
Gurgaon            24
Noida              21
Chennai            20
Hyderabad          20
Ahmedabad          14
Jaipur             13
Faridabad          13
Indore             12
Chandigarh         10
Vadodara            6
Aurangabad          5
Thane               5
Agra                5
Kolkata             5
Lucknow             5
Name: location, dtype: int64

### Domain Profile
(Does not require any additional cleaning)

In [34]:
n_domain_profiles = len(df.domain_profile.unique())
print(f"Number of domain_profile = {n_domain_profiles}")

df.domain_profile.value_counts()[-220:-200]

Number of domain_profile = 300


Wix Development                                1
React Native/React.js Development              1
3D Artist                                      1
System Development                             1
Environment/Level Design                       1
PHP/Laravel Development                        1
International Teaching (Coding)                1
Online Tutoring (Coding)                       1
Odoo Development                               1
Python Automation                              1
Robotics Research                              1
Backend Development (Node.js & Express.js)     1
React Native/Node.js Development               1
Technical Content Creation (YouTube Videos)    1
Information Technology (Team Lead)             1
Cyber Security Analyst                         1
Angular/Front-End Development                  1
Project Management (Web Design)                1
Raspberry Pi Application Development           1
Operating Systems Internals (Core OS)          1
Name: domain_profile

In [35]:
df.domain_profile.value_counts()[:20]

Web Development             422
Mobile App Development      150
Full Stack Development       24
Software Testing             23
WordPress Development        22
Python Development           22
Node.js Development          22
Backend Development          20
Flutter Development          20
PHP Development              17
Software Development         16
Product Management           15
Android App Development      13
Front End Development        11
Internet Of Things (IoT)     10
React Native Development      9
iOS App Development           9
ReactJs Development           9
Java Development              7
Angular Development           7
Name: domain_profile, dtype: int64

In [36]:
count_list

{189: ['PHP'],
 294: ['HTML'],
 267: ['CSS'],
 335: ['JavaScript'],
 149: ['Python'],
 128: ['WordPress'],
 62: ['AngularJS'],
 176: ['Node.js'],
 188: ['ReactJS'],
 102: ['MySQL'],
 60: ['jQuery'],
 29: ['CodeIgniter', 'Manual Testing', 'Linux'],
 9: ['NoSQL', 'Product Management'],
 61: ['LARAVEL'],
 106: ['REST API'],
 8: ['SQLite',
  'Embedded Systems',
  'Google Cloud Computing',
  'Cloud Computing',
  'Data Science',
  'Deep Learning'],
 103: ['React Native'],
 105: ['Android'],
 63: ['SQL'],
 88: ['MongoDB'],
 39: ['C++ Programming'],
 146: ['Java'],
 43: ['Angular 7.0', '.NET'],
 21: ['ASP.NET', 'Vue Js', 'User Interface (UI) Development', 'PostgreSQL'],
 25: ['MS SQL Server', 'JSON', 'Redux'],
 36: ['Amazon Web Services (AWS)'],
 7: ['Blockchain',
  'Unreal Engine',
  'Microsoft Visual Studio',
  'Docker',
  'Git Bash',
  'Apache',
  'Web Application Security'],
 14: ['Unity 3D', 'Firebase Cloud Messaging', 'MS-Office'],
 13: ['Unity Engine',
  'Adobe Illustrator',
  'C#.NET',

In [37]:
n_skills

{189: 1,
 294: 1,
 267: 1,
 335: 1,
 149: 1,
 128: 1,
 62: 1,
 176: 1,
 188: 1,
 102: 1,
 60: 1,
 29: 3,
 9: 2,
 61: 1,
 106: 1,
 8: 6,
 103: 1,
 105: 1,
 63: 1,
 88: 1,
 39: 1,
 146: 1,
 43: 2,
 21: 4,
 25: 3,
 36: 1,
 7: 7,
 14: 3,
 13: 5,
 2: 25,
 3: 14,
 17: 4,
 41: 1,
 20: 2,
 6: 9,
 79: 1,
 23: 3,
 27: 1,
 38: 1,
 24: 1,
 5: 13,
 30: 1,
 16: 2,
 77: 1,
 53: 1,
 19: 2,
 54: 1,
 1: 45,
 15: 3,
 34: 2,
 18: 2,
 31: 1,
 10: 3,
 4: 9,
 45: 1,
 12: 3,
 35: 1,
 11: 4,
 32: 1,
 28: 1}

### Dates

In [38]:
df.apply_by.unique()

array(["14 Jul' 21", "13 Jul' 21", "4 Jul' 21", "5 Jul' 21", "8 Jul' 21",
       "2 Jul' 21", "3 Jul' 21", "6 Jul' 21", "9 Jul' 21", "7 Jul' 21",
       "12 Jul' 21", "10 Jul' 21", "20 Jul' 21", "11 Jul' 21",
       "31 Jul' 21", "24 Jul' 21", "25 Jul' 21", "1 Jul' 21",
       "15 Jul' 21", "3 Aug' 21", "16 Jul' 21", "27 Jul' 21", "4 Aug' 21",
       "23 Jul' 21", "26 Jul' 21", "28 Jul' 21"], dtype=object)

### Merging dataset

In [41]:
m_df = pd.read_csv(f'internshala_clean.csv')
n_df = pd.read_csv(f'internshala_clean_june.csv')

In [45]:
combine_df = pd.concat([m_df, n_df])

In [46]:
combine_df[combine_df.duplicated()] 

,location,domain_profile,company_name,about_the_work_from_home_job/internship,skills,who_can_apply,other_requirements,perks,number_of_openings,company_details,...,Fashion Designing,Instagram Marketing,Interpersonal Skills,Odoo,OpenGL,PLC SCADA,Proteus Design Suite,Recruitment,Substance Painter,Voiceover
300,Work From Home,Web Development,LUshdicker,Selected intern's day-to-day responsibilities ...,PHP\nMySQL\nSearch Engine Optimization (SEO)\n...,Only those candidates can apply who:\n1. are a...,NaN,Certificate\n Flexible work hours,5,Website\nLUshdicker is an Indian online market...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
311,Work From Home,Mobile App Development,Compose Corner,Selected intern's day-to-day responsibilities ...,React Native\nReactJS\nFirebase,Only those candidates can apply who:\n1. are a...,NaN,Certificate\n Letter of recommendation\n Flexi...,1,Website\nCompose Corner is a collaborative pla...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
312,Work From Home,Technical Project Management,Zansys Technologies,Selected intern's day-to-day responsibilities ...,PHP\nSoftware Testing\nReactJS,Only those candidates can apply who:\n1. are a...,NaN,Certificate\n 5 days a week,1,Website\nZansys Technologies is a software ser...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
317,Surat,iOS App Development,Square Infosoft,NaN,Algorithms\nData Structures\nXcode\nSwift\niOS,Only those candidates can apply who:\n1. are a...,NaN,Certificate\n Letter of recommendation\n Flexi...,3,Website\nSquare Infosoft is a software develop...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
321,Kochi,Android App Development,CoinedOne Technologies Private Limited,NaN,XML\nSQLite\nAndroid\nKotlin,Only those candidates can apply who:\n1. are a...,1. Excellent in OOPS concepts\n2. Good working...,Certificate\n Letter of recommendation\n Infor...,2,Website\nEmpowering everyone to achieve more o...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2348,Kottakkal,Web Development,UCPro LLP,NaN,PHP\nHTML\nBootstrap\nReactJS\nVue Js,Only those candidates can apply who:\n1. are a...,NaN,Certificate\n Letter of recommendation\n Flexi...,1,Website\nWe are a team working in the ambiance...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2350,Work From Home,React Native Developer,QuadB,Selected intern's day-to-day responsibilities ...,NaN,Only those candidates can apply who:\n1. are a...,NaN,Certificate\n Letter of recommendation,4,Website\nQuadB is India's first completely cus...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2365,Work From Home,WordPress Development,Dhanam,Selected intern's day-to-day responsibilities ...,NaN,Only those candidates can apply who:\n1. are a...,NaN,Certificate\n Letter of recommendation\n Flexi...,1,We are building India's largest wealth tech co...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2372,Jaipur,Front End Development,Pingal IT Solutions,NaN,HTML\nCSS\nJavaScript\nAdobe Photoshop\nCorelD...,Only those candidates can apply who:\n1. are a...,NaN,Certificate\n Letter of recommendation\n 5 day...,2,LinkedIn page\nPingal IT Solutions is a global...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
combine_df.drop_duplicates(keep='first')

,location,domain_profile,company_name,about_the_work_from_home_job/internship,skills,who_can_apply,other_requirements,perks,number_of_openings,company_details,...,Fashion Designing,Instagram Marketing,Interpersonal Skills,Odoo,OpenGL,PLC SCADA,Proteus Design Suite,Recruitment,Substance Painter,Voiceover
0,Work From Home,Embedded Android Development,Augrade Private Limited,Selected intern's day-to-day responsibilities ...,Java\nLinux\nEmbedded Systems\nC Programming\n...,Only those candidates can apply who:\n1. are a...,1. Experience in hands-on development and trou...,Certificate\n Flexible work hours\n 5 days a week,2,"At Augrade, we work to rethink tomorrow and im...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Work From Home,Technical Project Management,Zansys Technologies,Selected intern's day-to-day responsibilities ...,PHP\nSoftware Testing\nReactJS,Only those candidates can apply who:\n1. are a...,NaN,Certificate\n 5 days a week,1,Website\nZansys Technologies is a software ser...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Work From Home,Full Stack Development (Web+Flutter),Rising Youth Foundation,Selected intern's day-to-day responsibilities ...,MongoDB\nNode.js\nFlutter\nREST API\nDart\nAnd...,Only those candidates can apply who:\n1. are a...,NaN,Certificate\n Letter of recommendation,1,Facebook page\nRising Youth Foundation is an A...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aurangabad,Web Development,44property,NaN,HTML\nCSS\nJavaScript\njQuery\nMS-Office\nWord...,Only those candidates can apply who:\n1. are a...,NaN,Certificate\n Letter of recommendation\n Flexi...,5,Website\n44property is a website dedicated to ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Work From Home,React Native Development,CogniAble,Selected intern's day-to-day responsibilities ...,HTML\nCSS\nReact Native\nRedux,Only those candidates can apply who:\n1. are a...,NaN,NaN,1,Website\nCogniAble is an organization working ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,Mumbai,Odoo Development,Best Roadways Limited,NaN,Odoo,Only those candidates can apply who:\n1. are a...,1. Must have strong knowledge of Odoo,Certificate,1,Website;Best Roadways was founded in 1985 with...,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1209,Work From Home,Web Development,Dot2Globe Solutions India Private Limited,Selected intern's day-to-day responsibilities ...,HTML\nCSS\nJavaScript\nUI & UX Design\nFigma,Only those candidates can apply who:\n1. are a...,1. Must have worked as UI/UX design intern (wi...,Certificate\n 5 days a week,1,Website;Dot2Globe Solutions is a start-up orga...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1210,Work From Home,Web Development,TikiLIVE,Selected intern's day-to-day responsibilities ...,NaN,Only those candidates can apply who:\n1. are a...,NaN,Certificate\n Letter of recommendation,3,Website;We make OTT delivery solutions worldwi...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1211,Work From Home,Web Development,Success Genix,Selected intern's day-to-day responsibilities ...,HTML\nCSS\nREST API\nReactJS\nAmazon Web Serve...,Only those candidates can apply who:\n1. are a...,NaN,Certificate\n Letter of recommendation\n Flexi...,2,Success Genix is a team of people that make ou...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
combine_df.to_csv("internshala_may_june.csv")